<h1>Document retrieval (Часть 2)</h1>

In [1]:
import os
import pandas as pd

os.chdir('../')

In [2]:
PATH_TO_PREPROCESSED = os.path.join('data', 'preprocessed')

df_train = pd.read_csv(os.path.join(PATH_TO_PREPROCESSED, 'questions_train.csv'))
df_val = pd.read_csv(os.path.join(PATH_TO_PREPROCESSED, 'questions_val.csv'))
df_test = pd.read_csv(os.path.join(PATH_TO_PREPROCESSED, 'questions_test.csv'))

df_train.head()

,id,question,context_id,preprocessed
0,62310,чем представлены органические остатки?,1,органический остаток
1,28101,что найдено в кремнистых сланцах железорудной ...,1,железорудный канада кремнистый сланцы формация
2,48834,что встречается в протерозойских отложениях?,1,отложение протерозойский
3,83056,что относится к числу древнейших растительных ...,1,остаток растительный древний
4,5816,как образовалось графито-углистое вещество?,1,вещество графито углистый


In [3]:
import concurrent.futures
import asyncio
from app.nlp.document_retrieval import perform_search

async def run_search(ix, queries):
    loop = asyncio.get_event_loop()
    results = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [
            loop.run_in_executor(executor, perform_search, ix, query) for query in queries
        ]
        for result in await asyncio.gather(*futures):
            results.append(result)
    return results

In [4]:
from app.database.main_database import DataRepository

dr = DataRepository()
ix = dr.get_index()

In [5]:
df_train['relevant'] = await run_search(ix, df_train['preprocessed'])

In [ ]:
df_train

,id,question,context_id,preprocessed,relevant
0,62310,чем представлены органические остатки?,1,органический остаток,"[11096, 1576, 12504, 1, 8005, 13270, 12729, 11..."
1,28101,что найдено в кремнистых сланцах железорудной ...,1,формация кремнистый канада сланцы железорудный,"[1, 5989, 9989, 9862, 4678, 218, 9737, 12796, ..."
2,48834,что встречается в протерозойских отложениях?,1,отложение протерозойский,"[1, 13075, 7120, 780, 901, 4472, 3312, 7713, 8..."
3,83056,что относится к числу древнейших растительных ...,1,остаток древний растительный,"[1, 843, 2, 12679, 10385, 9542, 11278, 9533, 1..."
4,5816,как образовалось графито-углистое вещество?,1,графито углистый вещество,"[1, 4723, 10189, 2237, 8660, 12314, 12394, 123..."
...,...,...,...,...,...
45200,6601,Когда подали заявку на патент на двигатель с ж...,9077,ленуар патент топливо заявка жидкий двигатель ...,"[9077, 4564, 220, 3544, 2115, 1255, 3860, 1083..."
45201,84192,Что создал после отклонения заявки Николаус Ау...,9077,отто николаус отклонение заявка аугуст,"[9077, 8625, 5939, 9381, 2692, 3974, 7751, 51,..."
45202,38284,Что было целью разыгранного Веберами спектакля?,9078,спектакль цель вебер,"[9078, 4559, 6675, 3355, 6862, 8511, 2689, 744..."
45203,73427,Что не мог подписать Моцарт из-за сильно разви...,9078,честь чувство развитый моцарт,"[9078, 6420, 11943, 8032, 7533, 2257, 9687, 18..."


In [9]:
df_train[df_train['relevant'].apply(len) == 0]

,id,question,context_id,preprocessed,relevant
1559,7997,Чем гистологически является пзазма?,282,пзазм,[]
5696,28308,Когда были открыты траспозоны?,1016,траспозон,[]
7432,8495,Как называлась битка в 1757 году,1333,биток,[]
7825,84134,Что такое динамтит?,1404,динамтит,[]
7942,74026,Какую систему хотели приспособить к трамбону?,1427,трамбон,[]
8832,14321,Как по-другому называют в билогии животнорасте...,1599,животнорастения билогия,[]
9106,5837,Для чего использовался вузузел?,1650,вузузел,[]
9204,75664,Кто сменил Ходдла?,1668,ходдло,[]
12673,5110,Какая существует наиболее распостраненная форм...,2300,розмножение распостраненный,[]
14998,62627,Что такое собственость?,2744,собственость,[]


In [13]:
def find_place(row: pd.DataFrame) -> int:
    real_id = row['context_id']
    lst = row['relevant']
    try:
        return lst.index(real_id)
    except ValueError:
        return -1

In [22]:

df_train[df_train.apply(find_place, axis=1) < 0]

,id,question,context_id,preprocessed,relevant
87,24794,Где был создан первый троллейбус?,15,троллейбус,"[2278, 3486, 11735, 1452, 10692, 622, 1828, 19..."
88,5797,В какой стране был создан первый троллейбус?,15,троллейбус,"[2278, 3486, 11735, 1452, 10692, 622, 1828, 19..."
89,68432,Кем был создан первый троллейбус?,15,троллейбус,"[2278, 3486, 11735, 1452, 10692, 622, 1828, 19..."
149,17232,Какие предприятия стали работать фактически за...,25,гг предприятие,"[4994, 12658, 9778, 10625, 10921, 4609, 8939, ..."
150,40695,Какие виды промышленности входят в пищевую отр...,25,экономика пищевой отрасль промышленность,"[509, 4704, 538, 4648, 4365, 4971, 7578, 44, 4..."
...,...,...,...,...,...
45160,23331,что характеризует функция,9063,функция,"[7840, 3007, 695, 4214, 6045, 12167, 12619, 15..."
45170,43443,На каком языке исполнялись джазовые композиции...,9067,джазовый композиция язык,"[8105, 779, 8838, 782, 2303, 8900, 517, 7931, ..."
45175,46090,Какой скрипач и флейтист одновременно также пр...,9069,скрипач группа флейтист,"[7453, 7584, 861, 3201, 243, 5184, 2523, 6959,..."
45196,67749,Какие ЛМП считаются самыми совершенными?,9075,лмп совершенный,"[9071, 13253, 8115, 6261, 6696, 7543, 12148, 3..."


In [10]:
from pyaspeller import YandexSpeller

speller = YandexSpeller(lang='ru')

In [12]:
speller.spelled('трамбон')

'тромбон'

Нааугментируем данных из этого вопроса, чтобы сымитировать поведение пользователя

In [ ]:
from whoosh.qparser import QueryParser
from whoosh import index

def post(self,search_string):
    from whoosh import query
    from whoosh.lang.wordnet import Thesaurus
    f = open("C:\\Users\\prakhar\\Downloads\\prolog\\wn_s.pl")
    t = Thesaurus.from_file(f)
    synonsm_stream = []
    stream_list = search_string.split(" ")
    for token in stream_list:
        m = t.synonyms(token)
        m.append(token)
        synonsm_stream.append(" OR ".join(m))
    synonym_string = " ".join(synonsm_stream)

    ix = index.open_dir("indexdir")
    with ix.searcher() as searcher:
        query = QueryParser("paragraph", ix.schema, termclass=query.Variations).parse(synonym_string)
        # "treatment of lung cancer in early stages"
        results = searcher.search(query, terms=True, )

<h3>Применение</h3>

In [ ]:
from app.nlp.document_retrieval import search_document
from app.database.models import async_session, get_index

In [ ]:
async with async_session() as session:
        ix = get_index("whoosh_index")  
        result = await search_document(ix, query_str)